# Modelo para Rusty Bargain

## Índice <a id='back'></a>
* [Introducción](#intro)
* [Etapa 1. Descripción y preprocesamiento de los datos](#data_review)
    * [1. 1. Información general de los datos](#data_review_data)
    * [1. 2. Valores problemáticos](#data_review_trouble)
    * [1. 3. Valores ausentes](#data_review_missing)
    * [1. 4. Conclusiones](#data_review_conclusions)
* [Etapa 2. Entrenamiento del modelo](#data_model)
    * [2. 1. Regresión lineal](#data_model_linear)
    * [2. 2. Árbol de decisión](#data_model_tree)
    * [2. 3. Bosque aleatorio](#data_model_forest)
    * [2. 4. CatBoost](#data_model_catboost)
    * [2. 5. LightGBM](#data_model_lightgbm)
    * [2. 6. Conclusiones](#data_model_conclusions)
* [Etapa 3. Análisis del modelo](#data_analysis)
* [Etapa 4. Conclusión general](#data_conclusion)

## Introducción <a id='intro'></a>

Rusty Bargain es un servicio de venta de coches de segunda mano que está desarrollando una app para atraer a nuevos clientes. Con la app, se puede averiguar rápidamente el valor de mercado de un coche. Se tiene acceso al historial, especificaciones técnicas, versiones de equipamiento y precios. El objetivo es crear un modelo que determine el valor de mercado, considerando los siguientes puntos:

- Calidad de la predicción.
- Velocidad de la predicción.
- Tiempo requerido para el entrenamiento.

Los datos se almacenan en el archivo `/datasets/car_dat.csv`. Como se desconoce la calidad de los datos, el proyecto consistirá en cuatro etapas:

1. Descripción y preprocesamiento de los datos.
2. Entrenamiento del modelo.
3. Análisis del modelo.
4. Conclusión general.

[Volver a Contenidos](#back)

## Etapa 1. Descripción y preprocesamiento de los datos <a id='data_review'></a>

Se importan las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import time

Se leerá el archivo y se guardará en la variable `data`. 

In [2]:
try:
    data = pd.read_csv('car_data.csv')
except:
    data = pd.read_csv('/datasets/car_data.csv')

### 1. 1. Información general de los datos <a id='data_review_data'></a>

Se imprimirá la información general de `data` y las primeras 10 filas de éste.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
data.head(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,0,33775,06/04/2016 19:17
6,01/04/2016 20:48,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,01/04/2016 00:00,0,67112,05/04/2016 18:18
7,21/03/2016 18:54,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,21/03/2016 00:00,0,19348,25/03/2016 16:47
8,04/04/2016 23:42,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,04/04/2016 00:00,0,94505,04/04/2016 23:42
9,17/03/2016 10:53,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,17/03/2016 00:00,0,27472,31/03/2016 17:17


El dataframe contiene 354,369 filas y 16 columnas, las cuales son:

1. `DateCrawled` - fecha en la que se descargó el perfil de la base de datos.
2. `Price` - precio en euros.
3. `VehicleType` - tipo de carrocería del vehículo.
4. `RegistrationYear` - año de matriculación del vehículo.
5. `Gearbox` - tipo de caja de cambios.
6. `Power` - potencia del vehículo (caballo de vapor, CV).
7. `Model` - modelo del vehículo.
8. `Mileage` - kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos).
9. `RegirtrationMonth` - mes de matriculación del vehículo.
10. `FuelType` - tipo de combustible.
11. `Brand` - marca del vehículo.
12. `NotRepaired` - vehículo con o sin reparación.
13. `DateCreated` - fecha de creación de perfil.
14. `NumberOfPictures` - cantidad de fotos del vehículo.
15. `PostalCode` - código postal del propietario del perfil (usuario).
16. `LastSeen` - fecha de la última vez que el usuario estuvo activo.

Se observa que las columnas no están nombradas en la convención estándar. Los datos de las columnas `DateCrawled`, `DateCreated` y `LastSeen` pueden ser cambiados a formato fecha. Existen valores ausentes en 5 columnas y al menos, la fila 7 contiene el valor 0 en el precio (columna que es el objetivo del modelo), es decir, pueden existir valores problemáticos en las columnas.

A continuación, se estandarizarán los nombres de las columnas.

In [5]:
data.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Mileage', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

In [6]:
data.columns = [
    'date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox', 'power', 'model', 'mileage',
    'registration_month', 'fuel_type', 'brand', 'not_repaired', 'date_created', 'number_of_pictures',
    'postal_code', 'last_seen'
]
data.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'mileage', 'registration_month', 'fuel_type', 'brand',
       'not_repaired', 'date_created', 'number_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

Se cambiará el formato de las tres columnas que contienen fechas.

In [7]:
for item in ['date_crawled', 'date_created', 'last_seen']:
    data[item] = pd.to_datetime(data[item], format='%d/%m/%Y %H:%M')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        354369 non-null  datetime64[ns]
 1   price               354369 non-null  int64         
 2   vehicle_type        316879 non-null  object        
 3   registration_year   354369 non-null  int64         
 4   gearbox             334536 non-null  object        
 5   power               354369 non-null  int64         
 6   model               334664 non-null  object        
 7   mileage             354369 non-null  int64         
 8   registration_month  354369 non-null  int64         
 9   fuel_type           321474 non-null  object        
 10  brand               354369 non-null  object        
 11  not_repaired        283215 non-null  object        
 12  date_created        354369 non-null  datetime64[ns]
 13  number_of_pictures  354369 no

Se han cambiado el formato de las columnas que contienen fechas. Estas columnas funcionan como identificadores únicos. Se imprimirá la cantidad de filas duplicadas.

In [8]:
print(data.duplicated().sum())

262


Esta cantidad no representa ni el 1% de la cantidad total de filas que se tienen. Por esta razón, se decide eliminarlas.

In [9]:
data = data.drop_duplicates().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354107 entries, 0 to 354106
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        354107 non-null  datetime64[ns]
 1   price               354107 non-null  int64         
 2   vehicle_type        316623 non-null  object        
 3   registration_year   354107 non-null  int64         
 4   gearbox             334277 non-null  object        
 5   power               354107 non-null  int64         
 6   model               334406 non-null  object        
 7   mileage             354107 non-null  int64         
 8   registration_month  354107 non-null  int64         
 9   fuel_type           321218 non-null  object        
 10  brand               354107 non-null  object        
 11  not_repaired        282962 non-null  object        
 12  date_created        354107 non-null  datetime64[ns]
 13  number_of_pictures  354107 no

En la siguiente sección se investigará valores problemáticos de las columnas y se decidirá el mejor tratamiento.

[Volver a Contenidos](#back)

### 1. 2. Valores problemáticos <a id='data_review_trouble'></a>

Primero se investigarán las columnas con datos numéricos.

In [10]:
print(data.describe())

               price  registration_year          power        mileage  \
count  354107.000000      354107.000000  354107.000000  354107.000000   
mean     4416.433287        2004.235355     110.089651  128211.811684   
std      4514.338584          90.261168     189.914972   37906.590101   
min         0.000000        1000.000000       0.000000    5000.000000   
25%      1050.000000        1999.000000      69.000000  125000.000000   
50%      2700.000000        2003.000000     105.000000  150000.000000   
75%      6400.000000        2008.000000     143.000000  150000.000000   
max     20000.000000        9999.000000   20000.000000  150000.000000   

       registration_month  number_of_pictures    postal_code  
count       354107.000000            354107.0  354107.000000  
mean             5.714182                 0.0   50507.145030  
std              3.726682                 0.0   25784.212094  
min              0.000000                 0.0    1067.000000  
25%              3.000000  

Las columnas con datos problemáticos son:
 - `price`, contiene filas con valor igual a 0.
 - `registration_year` contiene años que salen del rango de la historia del automóvil (de 1886 a 2023).
 - `registration_month` contiene filas con mes igual 0 (el rango es entre 1 y 12).

Se mostrarán los porcentajes de filas con valores problemáticas en estas dolunmas, con respecto a la cantidad total de las filas para decidir su tratamiento.

In [11]:
cnt_price = len(data[data['price'] == 0])
ratio_price = cnt_price / len(data)

print(f'Existen {cnt_price} filas con precio 0, esto representa un {ratio_price:.0%}')

Existen 10770 filas con precio 0, esto representa un 3%


Como solamente representan el 3% y además, es la columna objetivo del modelo, se decide eliminar estas filas.

In [12]:
data = data[data['price'] > 0]

A continuación se procede de manera similar con la columna `registration_year`.

In [13]:
cnt_year = len(data) - len(data[(data['registration_year'] >= 1886) & (data['registration_year'] <= 2023)])
ratio_year = cnt_year / len(data)

print(f'Existen {cnt_year} filas con años absurdos, esto representa un {ratio_year:.0%}')

Existen 139 filas con años absurdos, esto representa un 0%


Se eliminarán estas filas, pues representan menos del 1%.

In [14]:
data = data[(data['registration_year'] >= 1886) & (data['registration_year'] <= 2023)]

Se procede de manera similar con la columna `registration_month`.

In [15]:
cnt_month = len(data[data['registration_month'] == 0])
ratio_month = cnt_month / len(data)

print(f'Existen {cnt_month} filas con mes igual a 0, esto representa un {ratio_month:.0%}')

Existen 32768 filas con mes igual a 0, esto representa un 10%


Esta filas representan un 10% y no hay manera de saber el mes de registro por lo que se decide elimnarlas.

In [16]:
data = data[data['registration_month'] > 0].reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310430 entries, 0 to 310429
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        310430 non-null  datetime64[ns]
 1   price               310430 non-null  int64         
 2   vehicle_type        289230 non-null  object        
 3   registration_year   310430 non-null  int64         
 4   gearbox             300907 non-null  object        
 5   power               310430 non-null  int64         
 6   model               298168 non-null  object        
 7   mileage             310430 non-null  int64         
 8   registration_month  310430 non-null  int64         
 9   fuel_type           292326 non-null  object        
 10  brand               310430 non-null  object        
 11  not_repaired        263706 non-null  object        
 12  date_created        310430 non-null  datetime64[ns]
 13  number_of_pictures  310430 no

Ahora se imprimirán los valores únicos de las columnas con datos categóricos. La primera columna es `vhicle_type`.

In [17]:
categorical = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']

for item in categorical:
    print(item)
    print(data[item].sort_values().unique())
    print()

vehicle_type
['bus' 'convertible' 'coupe' 'other' 'sedan' 'small' 'suv' 'wagon' nan]

gearbox
['auto' 'manual' nan]

model
['100' '145' '147' '156' '159' '1_reihe' '1er' '200' '2_reihe' '300c'
 '3_reihe' '3er' '4_reihe' '500' '5_reihe' '5er' '601' '6_reihe' '6er'
 '7er' '80' '850' '90' '900' '9000' '911' 'a1' 'a2' 'a3' 'a4' 'a5' 'a6'
 'a8' 'a_klasse' 'accord' 'agila' 'alhambra' 'almera' 'altea' 'amarok'
 'antara' 'arosa' 'astra' 'auris' 'avensis' 'aveo' 'aygo' 'b_klasse'
 'b_max' 'beetle' 'berlingo' 'bora' 'boxster' 'bravo' 'c1' 'c2' 'c3' 'c4'
 'c5' 'c_klasse' 'c_max' 'c_reihe' 'caddy' 'calibra' 'captiva' 'carisma'
 'carnival' 'cayenne' 'cc' 'ceed' 'charade' 'cherokee' 'citigo' 'civic'
 'cl' 'clio' 'clk' 'clubman' 'colt' 'combo' 'cooper' 'cordoba' 'corolla'
 'corsa' 'cr_reihe' 'croma' 'crossfire' 'cuore' 'cx_reihe' 'defender'
 'delta' 'discovery' 'doblo' 'ducato' 'duster' 'e_klasse' 'elefantino'
 'eos' 'escort' 'espace' 'exeo' 'fabia' 'fiesta' 'focus' 'forester'
 'forfour' 'fortwo' 'fo

No existen duplicados implícitos en ninguna de las columnas categóricas. En la siguiente sección se tratarán los valores ausentes.

[Volver a Contenidos](#back)

### 1. 3. Valores ausentes <a id='data_review_missing'></a>

Se imprimirá la información sobre los valores ausentes.

In [18]:
print(data.isna().sum())

date_crawled              0
price                     0
vehicle_type          21200
registration_year         0
gearbox                9523
power                     0
model                 12262
mileage                   0
registration_month        0
fuel_type             18104
brand                     0
not_repaired          46724
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64


La columna `not_repaired` es la que contiene una mayor cantidad de valores ausentes y no es posible determinar esta información a partir de las otras columnas, por lo que se rellenarán estos valores con `'unknown'`.

In [19]:
data['not_repaired'] = data['not_repaired'].fillna('unknown')
print(data.isna().sum())

date_crawled              0
price                     0
vehicle_type          21200
registration_year         0
gearbox                9523
power                     0
model                 12262
mileage                   0
registration_month        0
fuel_type             18104
brand                     0
not_repaired              0
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64


Ahora la columna con mayor cantidad de valores ausentes es `vehicle_type`, estas filas representan casi el 7% del tamaño del dataframe. Las otras columnas son `gearbox`, `model` y `fuel_type`, las cuales no se pueden determinar con las datos que se tienen. Por esta razón, se decide elimnar las filas que contienen valores ausentes.

In [20]:
data = data.dropna().reset_index(drop=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265967 entries, 0 to 265966
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        265967 non-null  datetime64[ns]
 1   price               265967 non-null  int64         
 2   vehicle_type        265967 non-null  object        
 3   registration_year   265967 non-null  int64         
 4   gearbox             265967 non-null  object        
 5   power               265967 non-null  int64         
 6   model               265967 non-null  object        
 7   mileage             265967 non-null  int64         
 8   registration_month  265967 non-null  int64         
 9   fuel_type           265967 non-null  object        
 10  brand               265967 non-null  object        
 11  not_repaired        265967 non-null  object        
 12  date_created        265967 non-null  datetime64[ns]
 13  number_of_pictures  265967 no

No existen valores ausentes.

[Volver a Contenidos](#back)

### 1. 4. Conclusiones <a id='data_review_conclusions'></a>

En esta etapa se abrió el archivo y se guardó en un dataset, el cual contiene información del automóvil como precio, tipo, año y mes de registro, tipo de transmisión, potencia, modelo, kilometraje, tipo de combustible, marca y si ha sido reparado o no. También contiene información del usuario como las fechas en las que se creo el perfil, última vez de actividad o en la que se descargaron los datos.

Se cambió el formato de los nombres de las columnas a la manera convencional (minúsculas y con space case). Se cambió el formato de las columnas con fechas, se eliminaron filas duplicadas y se trataron los valores ausentes. Los valores ausentes de la columna con la información de posibles reparaciones se reemplazaron con `'unknown'` y se eliminaron todos las filas con valores ausentes en las columnas restantes.

En la siguiente etapa se entrenarán modelos y se buscará el de mejor calidad.

[Volver a Contenidos](#back)

## Etapa 2. Entrenamiento del modelo <a id='data_model'></a>

En esta etapa se entrenarán modelos con 5 diferentes algoritmos: regresión lineal, árbol de decisión, bosque aleatorio, CatBoost y LightGBM.

La columna objetivo del modelo es la columna `price` y las columnas que no aportan información para entrenar a los modelos son:
- `date_crawled` - información de usuario no del auto.
- `date_created` - información de usuario no del auto.
- `number_of_pictures` - sin utilidad.
- `postal_code` _ información de usuario no del auto.
- `last_seen` - información de usuario no del auto.

Así, las características son las columnas restantes.

In [21]:
features = data.drop(['price', 'date_crawled', 'date_created', 'number_of_pictures', 'postal_code', 'last_seen'], axis=1)
target = data['price']

features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265967 entries, 0 to 265966
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   vehicle_type        265967 non-null  object
 1   registration_year   265967 non-null  int64 
 2   gearbox             265967 non-null  object
 3   power               265967 non-null  int64 
 4   model               265967 non-null  object
 5   mileage             265967 non-null  int64 
 6   registration_month  265967 non-null  int64 
 7   fuel_type           265967 non-null  object
 8   brand               265967 non-null  object
 9   not_repaired        265967 non-null  object
dtypes: int64(4), object(6)
memory usage: 20.3+ MB


Para los modelos basados en regresión lineal, árbol de decisión y bosque aleatorio, es necesario codificar las variables categóricas.

In [22]:
categorical = ['vehicle_type', 'gearbox', 'model', 'registration_month','fuel_type', 'brand', 'not_repaired']

ohe = pd.get_dummies(data = features, columns = categorical, drop_first=True)

De esta manera, se pueden dividir los datos en conjunto de entrenamiento y de prueba, con una proporción de 4:1. Como los otros modelos no necesitan la codificación de las variables categóricas, entonces también se dividirá sin la codificación.

In [23]:
features_train_ohe, features_test_ohe, target_train_ohe, target_test_ohe = train_test_split(ohe, target, test_size=0.2, random_state=54321)
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=54321)

print('Tamaño de features_train_ohe:', features_train_ohe.shape)
print('Tamaño de features_test_ohe:', features_test_ohe.shape)
print('Tamaño de target_train_ohe:', target_train_ohe.shape)
print('Tamaño de target_test_ohe:', target_test_ohe.shape)
print()
print('Tamaño de features_train:', features_train.shape)
print('Tamaño de features_test:', features_test.shape)
print('Tamaño de target_train:', target_train.shape)
print('Tamaño de target_test:', target_test.shape)

Tamaño de features_train_ohe: (212773, 316)
Tamaño de features_test_ohe: (53194, 316)
Tamaño de target_train_ohe: (212773,)
Tamaño de target_test_ohe: (53194,)

Tamaño de features_train: (212773, 10)
Tamaño de features_test: (53194, 10)
Tamaño de target_train: (212773,)
Tamaño de target_test: (53194,)


También se estandarizarán las variables numéricas.

In [24]:
numerics = ['registration_year', 'power', 'mileage']

scaler = StandardScaler()
scaler.fit(features_train_ohe[numerics])

features_train[numerics] = scaler.transform(features_train[numerics])
features_test[numerics] = scaler.transform(features_test[numerics])

features_train_ohe[numerics] = scaler.transform(features_train_ohe[numerics])
features_test_ohe[numerics] = scaler.transform(features_test_ohe[numerics])

Se utilizará la métrica RECM para evaluar la calidad de los modelos, por ello se creará la función, utilizando la función `mean_squared_error`.

In [25]:
def rmse(y_true, y_predict):
    mse = mean_squared_error(y_true, y_predict)
    return mse ** 0.5

rmse_scorer = make_scorer(rmse, greater_is_better=False)

Finalmente, se creará una tabla para registrar la información resultante de entrenar los modelos y así poder compararlos.

In [26]:
model_info = pd.DataFrame(columns=('model','fit_time','predict_time','rmse'))
model_info.head()

,model,fit_time,predict_time,rmse


Ya se tiene lo necesario para comenzar el entrenamiento de los modelos.

[Volver a Contenidos](#back)

### 2. 1. Regresión lineal <a id='data_model_linear'></a>

El primer modelo a entrenar será basado en la regresión lineal. Este modelo se utilizará como la métrica RECM a superar.

Se realizará validación cruzada.

In [27]:
lr_model = LinearRegression()
lr_score=cross_val_score(lr_model, features_train_ohe, target_train_ohe, scoring=rmse_scorer, cv=5)
print(lr_score.mean())

-2674.935711842474


Se entrenará al modelo y se tomará el tiempo, así como el tiempo de predicción y se calculará el RECM para adjuntar los datos en la tabla.

In [28]:
#fila con la información del modelo
lr_info = ['Linear Regression']

#Tomar tiempo de entrenamiento
start = time.time()
lr_model.fit(features_train_ohe, target_train_ohe)
end=time.time()

#agregar el tiempo de entrenamiento
lr_info.append(end - start)

#Tomar tiempo de predicción
start = time.time()
lr_predict = lr_model.predict(features_test_ohe)
end=time.time()

#agregar el tiempo de predicción
lr_info.append(end - start)

#RMSE
lr_rmse = rmse(target_test_ohe, lr_predict)

#add to row
lr_info.append(lr_rmse)

model_info.loc[len(model_info)] = lr_info
model_info.head()

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915


Se analizará si con otros modelos se puede obtener un RECM menor.

[Volver a Contenidos](#back)

### 2. 2. Árbol de decisión <a id='data_model_tree'></a>

A continuación se utilizará el algoritmo de árbol de decisión y se entrenarán modelos variando la profundidad y evaluando con validación cruzada.

In [29]:
for depth in range(1, 21):
    dt_model = DecisionTreeRegressor(max_depth=depth, random_state=54321)
    dt_score = cross_val_score(dt_model, features_train_ohe, target_train_ohe, scoring=rmse_scorer, cv=5)
    print('Max_depth', depth, 'score:', dt_score.mean())

Max_depth 1 score: -3577.4451257216883
Max_depth 2 score: -3135.405101217395
Max_depth 3 score: -2789.976861087192
Max_depth 4 score: -2546.9779781927123
Max_depth 5 score: -2387.775782323587
Max_depth 6 score: -2273.2288046255053
Max_depth 7 score: -2177.0671578053516
Max_depth 8 score: -2096.859174723884
Max_depth 9 score: -2033.9497795458458
Max_depth 10 score: -1981.8721848843575
Max_depth 11 score: -1942.935063562259
Max_depth 12 score: -1908.8567839310635
Max_depth 13 score: -1883.7408241896642
Max_depth 14 score: -1869.1147979669108
Max_depth 15 score: -1860.476417129131
Max_depth 16 score: -1859.4763123377295
Max_depth 17 score: -1859.0264029100624
Max_depth 18 score: -1866.0778038267467
Max_depth 19 score: -1873.20719374104
Max_depth 20 score: -1879.6643962929277


El mejor modelo es con profundidad máximas igual a 17, de esta manera se medirá el tiempo de entrenamiento y predicción con esta profundidad.

In [30]:
#fila con la información del modelo
dt_info = ['Decision Tree']

dt_model = DecisionTreeRegressor(max_depth=17, random_state=54321)

#Tomar tiempo de entrenamiento
start = time.time()
dt_model.fit(features_train_ohe, target_train_ohe)
end=time.time()

#agregar el tiempo de entrenamiento
dt_info.append(end - start)

#Tomar tiempo de predicción
start = time.time()
dt_predict = dt_model.predict(features_test_ohe)
end=time.time()

#agregar el tiempo de predicción
dt_info.append(end - start)

#RMSE
dt_rmse = rmse(target_test_ohe, dt_predict)

#add to row
dt_info.append(dt_rmse)

model_info.loc[len(model_info)] = dt_info
model_info.head()

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915
1,Decision Tree,2.387368,0.031758,1834.772587


Este modelo ha mejorado tanto los tiempos como la métrica RECM del modelo basado en la regresión lineal.

[Volver a Contenidos](#back)

### 2. 3. Bosque aleatorio <a id='data_model_forest'></a>

A continuación se utilizará el algoritmo de bosque aleatorio y se entrenarán modelos variando la profundidad y evaluando con validación cruzada. Se considerarán 50 árboles.

In [31]:
#Previamente se utilizó un rango de 1 a 26 pero el proceso tardaba más de 2 horas
for depth in range(24, 26):
    rf = RandomForestRegressor(n_estimators=50, max_depth=depth, random_state=54321)
    rf_score = cross_val_score(rf, features_train_ohe, target_train_ohe, scoring=rmse_scorer, cv=5)
    print('Max_depth', depth, 'score:', rf_score.mean())

Max_depth 24 score: -1571.4557267767864
Max_depth 25 score: -1570.8727012449758


El mejor modelo es con profundidad igual a 25 aunque es muy probable que se obtenga un mejor puntaje a mayor profundidad, sin embargo, el tiempo de entrenamiento es cada vez más lento. Por ello se tomará este parámetro.

In [32]:
#fila con la información del modelo
rf_info = ['Random Forest']

rf_model = RandomForestRegressor(n_estimators=50, max_depth=25, random_state=54321)

#Tomar tiempo de entrenamiento
start = time.time()
rf_model.fit(features_train_ohe, target_train_ohe)
end=time.time()

#agregar el tiempo de entrenamiento
rf_info.append(end - start)

#Tomar tiempo de predicción
start = time.time()
rf_predict = rf_model.predict(features_test_ohe)
end=time.time()

#agregar el tiempo de predicción
rf_info.append(end - start)

#RMSE
rf_rmse = rmse(target_test_ohe, rf_predict)

#add to row
rf_info.append(rf_rmse)

model_info.loc[len(model_info)] = rf_info
model_info.head()

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915
1,Decision Tree,2.387368,0.031758,1834.772587
2,Random Forest,85.885852,0.521819,1560.465073


Este modelo tiene el menor RECM pero es el que ha registrado mayor tiempo tanto de entrenamiendo como de predicción.

[Volver a Contenidos](#back)

### 2. 4. CatBoost <a id='data_model_catboost'></a>

Se utilizará CatBoostRegressor para entrenar al modelo. Este algoritmo no necesita la codificación de las variables categóricas. Se utilizarán diferentes hiperparámetros y se encontrarán los mejores utilizando GridSearchCV.

In [33]:
cbr_model = CatBoostRegressor()

#diccionario de parámetros
parameters = {
    'depth' : [12, 14, 16],
    'learning_rate' : [0.05, 0.1],
    'iterations' : [50, 100]
}

#bucles para encontrar los mejores hiperparámetros
grid = GridSearchCV(estimator=cbr_model, param_grid=parameters, scoring=rmse_scorer, cv=3, n_jobs=-1, verbose=0)

#entrenamiento sin codificación
grid.fit(features_train, target_train, cat_features=categorical)

#Mejores parámetros
print('Resultados de GridSearch')
print("\n La mejor puntuación:", grid.best_score_)
print("\n Los mejores parámetros:\n", grid.best_params_)

best_param=grid.best_params_

0:	learn: 4485.5645162	total: 156ms	remaining: 7.65s
0:	learn: 4497.4043066	total: 208ms	remaining: 10.2s
1:	learn: 4322.9887987	total: 291ms	remaining: 6.97s
0:	learn: 4314.2564331	total: 232ms	remaining: 11.4s
1:	learn: 4336.0033555	total: 331ms	remaining: 7.93s
0:	learn: 4320.0193883	total: 243ms	remaining: 11.9s
0:	learn: 4489.9098061	total: 275ms	remaining: 13.5s
2:	learn: 4170.5384597	total: 443ms	remaining: 6.95s
0:	learn: 4327.2260118	total: 171ms	remaining: 8.39s
1:	learn: 4012.1549845	total: 385ms	remaining: 9.23s
2:	learn: 4184.9662865	total: 488ms	remaining: 7.64s
0:	learn: 4485.5645162	total: 255ms	remaining: 25.2s
1:	learn: 4019.4832554	total: 445ms	remaining: 10.7s
1:	learn: 4327.7120209	total: 496ms	remaining: 11.9s
0:	learn: 4489.9098061	total: 282ms	remaining: 28s
1:	learn: 4027.8279876	total: 326ms	remaining: 7.83s
3:	learn: 4027.0300044	total: 627ms	remaining: 7.21s
2:	learn: 3746.6241966	total: 571ms	remaining: 8.95s
3:	learn: 4040.7480353	total: 682ms	remaining: 7

Con estos parámetros se entrenará el modelo y se tomará el tiempo.

In [34]:
#fila con la información del modelo
cbr_info = ['CatBoostRegressor']

cbr_model = CatBoostRegressor(
    depth=best_param['depth'],
    iterations=best_param['iterations'],
    learning_rate=best_param['learning_rate']
)

#Tomar tiempo de entrenamiento
start = time.time()
cbr_model.fit(features_train, target_train, cat_features=categorical, verbose=False, plot=False)
end=time.time()

#agregar el tiempo de entrenamiento
cbr_info.append(end - start)

#Tomar tiempo de predicción
start = time.time()
cbr_predict = cbr_model.predict(features_test)
end=time.time()

#agregar el tiempo de predicción
cbr_info.append(end - start)

#RMSE
cbr_rmse = rmse(target_test, cbr_predict)

#add to row
cbr_info.append(cbr_rmse)

model_info.loc[len(model_info)] = cbr_info
model_info.head()

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915
1,Decision Tree,2.387368,0.031758,1834.772587
2,Random Forest,85.885852,0.521819,1560.465073
3,CatBoostRegressor,27.360088,0.033150,1572.943566


Este modelo tiene un RECM mayor que el obtenido con el bosque aleatorio, sin embargo sus tiempos son menores.

[Volver a Contenidos](#back)

### 2. 5. LightGBM <a id='data_model_lightgbm'></a>

En esta sección se utilizará LightGBM para entrenar el modelo. Lo primero que se debe hacer es convertir las variables categóricas al tipo correcto.

In [35]:
for item in categorical:
    features_train[item] = features_train[item].astype('category')
    features_test[item] = features_test[item].astype('category')

Así como con el anterior, se utilizarán diferentes parámetros y con GridSearchCV se encontrarán los mejores.

In [36]:
model=LGBMRegressor()

parameters = {
    'num_leaves': [40, 50, 60],
    'learning_rate': [0.5, 0.1],
    'n_estimators': [50, 60,70]
}

grid = GridSearchCV(estimator=model, param_grid=parameters, scoring=rmse_scorer, cv=3, n_jobs=-1)
grid.fit(features_train, target_train)
best_param=grid.best_params_

print('Resultados de GridSearch')
print("\n La mejor puntuación:", grid.best_score_)
print("\n Los mejores parámetros:\n", grid.best_params_)

Resultados de GridSearch

 La mejor puntuación: -1578.17002080581

 Los mejores parámetros:
 {'learning_rate': 0.1, 'n_estimators': 70, 'num_leaves': 60}


Con estos parámetros se entrenará el modelo y se tomará el tiempo.

In [37]:
#fila con la información del modelo
lgbm_info = ['LightGBMRegressor']

lgbm_model = LGBMRegressor(
    learning_rate=best_param['learning_rate'],
    n_estimators=best_param['n_estimators'],
    num_leaves=best_param['num_leaves']
)

#Tomar tiempo de entrenamiento
start = time.time()
lgbm_model.fit(features_train, target_train)
end=time.time()

#agregar el tiempo de entrenamiento
lgbm_info.append(end - start)

#Tomar tiempo de predicción
start = time.time()
lgbm_predict = lgbm_model.predict(features_test)
end=time.time()

#agregar el tiempo de predicción
lgbm_info.append(end - start)

#RMSE
lgbm_rmse = rmse(target_test, lgbm_predict)

#add to row
lgbm_info.append(lgbm_rmse)

model_info.loc[len(model_info)] = lgbm_info
model_info.head()

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915
1,Decision Tree,2.387368,0.031758,1834.772587
2,Random Forest,85.885852,0.521819,1560.465073
3,CatBoostRegressor,27.360088,0.033150,1572.943566
4,LightGBMRegressor,0.398182,0.053457,1569.640920


Este modelo tiene menor tiempo de entrenamiento aunque el RECM es mayor que el de bosque aleatorio.

[Volver a Contenidos](#back)

### 2. 6. Conclusiones <a id='data_model_conclusions'></a>

En esta etapa se entrenaron modelos usando los algoritmos: regresión lineal, árbol de decisión, bosque aleatorio, CatBoost y LightGBM.

Se utilizaron diferentes hiperparámetros para cada uno de ellos y utilizando validación cruzada, o bien, GridSearchCV, se decidió cuales eran los mejores.

Con esto parámetros se entrenaron los diferentes modelos y se tomaron los tiempos tanto de entrenamiento como de predicción. La métrica de evaluación fue el RECM.

[Volver a Contenidos](#back)

## Etapa 3. Análisis del modelo <a id='data_analysis'></a>

Se imprimirá nuevamente la información de los modelos, obtenida en la etapa anterior.

In [38]:
model_info

,model,fit_time,predict_time,rmse
0,Linear Regression,2.791132,0.027558,2688.063915
1,Decision Tree,2.387368,0.031758,1834.772587
2,Random Forest,85.885852,0.521819,1560.465073
3,CatBoostRegressor,27.360088,0.033150,1572.943566
4,LightGBMRegressor,0.398182,0.053457,1569.640920


Tenemos que el modelo con el menor RECM es el entrenado con el algoritmo del bosque aleatorio, mientras que la regresión lineal resulta en el mayor valor de RECM. Sin embargo, este modelo funciona como una prueba de cordura.

Por otro lado, el bosque aleatorio es el que presenta mayor tiempo de entrenamiento, de hecho, la diferencia es muy grande comparado con todos lode demás modelos.

El modelo basado en LightGBMRegressor es el que presenta el mejor tiempo de entrenamiento y aunque ocupa el cuarto lugar con respecto al tiempo de predicción, la diferencia es mínima con respecto a los primeros tres lugares. Además, es el modelo que ocupa el segundo lugar con respecto al RECM (de menor a mayor).

Por lo tanto, el modelo basado en LightGBMRegressor es el mejor pues registra buenas puentuaciones tanto en RECM como en tiempos.

[Volver a Contenidos](#back)

## Etapa 4. Conclusión general <a id='data_conclusion'></a>

En la primera etapa se preprocesaron los datos, en primer lugar se renombraron las columnas del dataset conforme a la nomenclatura convencional y se cambiaron los datos correspondientes a formato fecha. Se reemplazron valores ausentes en la columna sobre la información de reparaciones con `'unknown'` y se eliminaron las filas que tenían valores ausentes en las demás columnas.

En la segunda etapa se entrenaron modelos utilizando cinco algoritmos diferentes. Se variaron los hiperpárametros y con validación cruzada o con GridSearchCV se elegieron los mejores. También se midieron los tiempos de entrenamiento y predicción.

En la tercera etapa se analizó la calidad de los modelos, tomando en cuenta tanto la métrica RECM como los tiempos obtenidos en la segunda etapa. El modelo con las mejores puntuaciones en cuanto a tiempo fue el realizado con LightGBMRegressor. Además, registró el segundo mejor valor de RECM. Por esta razón se concluyó que es el modelo más apropiado tomando en cuenta los requerimientos del cliente.

[Volver a Contenidos](#back)